In [2]:
import pandas as pd
import base64
import json

# Carregar dados do arquivo JSON
with open('dados.json', 'r') as f:
    config = json.load(f)

# Acessar as variáveis
data_inicio = config.get('data_inicio')
data_fim = config.get('data_fim')
data_inicio_antiga = config.get('data_inicio_antiga')
data_fim_antiga = config.get('data_fim_antiga')
print(f"Atual: {data_inicio} a {data_fim}")  
print(f"Antiga: {data_inicio_antiga} a {data_fim_antiga}")  

# Processar arquivos
if 'tabela' in config['files']:
    tabela = pd.read_excel(pd.io.common.BytesIO(base64.b64decode(config['files']['tabela'])))
    #print(tabela.head(1))

if 'tabela_projecao' in config['files']:
    tabela_projecao = pd.read_excel(pd.io.common.BytesIO(base64.b64decode(config['files']['tabela_projecao'])))
    #print(tabela_projecao.head(1))

if 'credenciais' in config['files']:
    credenciais_utilizadas = pd.read_excel(pd.io.common.BytesIO(base64.b64decode(config['files']['credenciais'])))
    #print(credenciais_utilizadas.head(1))

if 'report' in config['files']:
    report_de_uso = pd.read_excel(pd.io.common.BytesIO(base64.b64decode(config['files']['report'])))
    #print(report_de_uso.head(1))


Atual: 2024-07-08 a 2024-07-14
Antiga: 2024-07-01 a 2024-07-07


c:\Users\saman\Documents\Pastas_Jus_Analise\Fup_Automatico\novo_venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell R10361 is marked as a date but the serial value 6948614.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\saman\Documents\Pastas_Jus_Analise\Fup_Automatico\novo_venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\saman\Documents\Pastas_Jus_Analise\Fup_Automatico\novo_venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\saman\Documents\Pastas_Jus_Analise\Fup_Automatico\novo_venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl

○ Quantidade de credenciais

In [3]:
credenciais_utilizadas_df = pd.DataFrame(credenciais_utilizadas)

# Somar todos os números na coluna "number_of_credentials"
total_credenciais = credenciais_utilizadas_df['number_of_credentials'].sum()




print(f'● No período analisado, utilizamos {total_credenciais} credenciais diferentes (únicas) ')

● No período analisado, utilizamos 9537 credenciais diferentes (únicas) 


In [4]:
#Converte a coluna "Data de bloqueio" para o tipo datetime
tabela['Data do bloqueio'] = pd.to_datetime(tabela['Data do bloqueio'], format='%d/%m/%Y')

# Filtre o DataFrame para o intervalo de datas
tabela_filtrada = tabela[(tabela['Data do bloqueio'] >= data_inicio) & (tabela['Data do bloqueio'] <= data_fim)]
tabela_filtrada = pd.DataFrame(tabela_filtrada)

# Remover duplicatas na coluna "ID Credencial"
tabela_unica = tabela_filtrada.drop_duplicates(subset='ID Credencial')

# Obter a quantidade de linhas únicas
quantidade_linhas_unicas = tabela_unica.shape[0]

porcentagem_credenciais = round((quantidade_linhas_unicas*100)/total_credenciais, 2)

# Exibir o resultado
print(f'● {quantidade_linhas_unicas} credenciais diferentes receberam alerta BLOCKED pelo KM, representando {porcentagem_credenciais} do total de credenciais utilizadas')

● 283 credenciais diferentes receberam alerta BLOCKED pelo KM, representando 2.97 do total de credenciais utilizadas


In [5]:
quantidade_linhas_domains = credenciais_utilizadas_df['domain_alias'].count()


print(f"● {quantidade_linhas_domains} domains requisitaram credenciais")

● 93 domains requisitaram credenciais


In [6]:
# Filtrar os 3 maiores valores na coluna "number_of_credentials"
n_maiores = 10
maiores_credentials = credenciais_utilizadas_df.nlargest(n_maiores, 'number_of_credentials')

# Selecionar as colunas "domain_alias" e "number_of_credentials"
resultado = maiores_credentials[['domain_alias', 'number_of_credentials']]

# Exibir o resultado
print('Domain aliases com os maiores valores de number_of_credentials:')
print(resultado)

Domain aliases com os maiores valores de number_of_credentials:
    domain_alias  number_of_credentials
0     TJSC-EPROC                    640
1   TJSP-ESAJ-1G                    509
2       TRT4-PJE                    489
3      TRT15-PJE                    483
4    TJMG-PJE-1G                    443
5    TJMG-PJE-2G                    422
6      TRT10-PJE                    421
7  TJTO-EPROC-2G                    384
8       TRT6-PJE                    384
9  TJTO-EPROC-1G                    376


○ Buscas e sessões

In [7]:
report_de_uso_df = pd.DataFrame(report_de_uso)
total_buscas = report_de_uso_df['credential_count'].sum()

total_realizadas = report_de_uso_df['report_count'].sum()





print(f"● Nossos robôs buscaram {total_buscas} vezes por credenciais e com elas realizaram {total_realizadas} sessões de uso para o período analisado [ref]")


● Nossos robôs buscaram 70110 vezes por credenciais e com elas realizaram 2652081 sessões de uso para o período analisado [ref]


In [8]:
# Filtrar o DataFrame para manter apenas as linhas onde o status é 'success'
filtrado_success = report_de_uso_df[report_de_uso_df['status'] == 'success']

total_buscas_success = filtrado_success['report_count'].sum()

relação_ao_total_success= round((total_buscas_success*100)/ total_realizadas, 2)

print(f"○ {total_buscas_success} sessões tiveram sucesso, correspondendo a {relação_ao_total_success} do total")

○ 2401238 sessões tiveram sucesso, correspondendo a 90.54 do total


In [9]:
filtrado_error = report_de_uso_df[
    (report_de_uso_df['status'] == 'error') | 
    (report_de_uso_df['status'] == 'two_factor_error')
]

total_buscas_error = filtrado_error['report_count'].sum()
relação_ao_total_error = round((total_buscas_error*100)/ total_realizadas, 2)

print(f"○ {total_buscas_error} sessões tiveram erros, correspondendo a {relação_ao_total_error} do total.")

○ 250843 sessões tiveram erros, correspondendo a 9.46 do total.


In [10]:
filtrado_two_factor = report_de_uso_df[report_de_uso_df['status'] == 'two_factor_error']

total_buscas_two_factor = filtrado_two_factor['report_count'].sum()




print(f"■ Dessas sessões com erro, tivemos {total_buscas_two_factor} sessões com erro no 2FA para:")

■ Dessas sessões com erro, tivemos 0 sessões com erro no 2FA para:


In [11]:
filtrado_two_factor_pd = pd.DataFrame(filtrado_two_factor)

# Listar todos os valores da coluna "domain_alias"
valores_domain_alias = filtrado_two_factor_pd['domain_alias'].drop_duplicates()


print(valores_domain_alias)


Series([], Name: domain_alias, dtype: object)


In [12]:
quantidade_linhas_blocked = len(tabela_filtrada)

print(f"■ Tivemos {quantidade_linhas_blocked} alertas BLOCKED de credenciais realizados pelo KM [ref]")

■ Tivemos 361 alertas BLOCKED de credenciais realizados pelo KM [ref]


In [13]:
relação_blocked_total = round((quantidade_linhas_blocked*100)/total_buscas,2)


print(f"■ O volume de credenciais com alerta BLOCKED representa {relação_blocked_total} do volume total de buscas por credenciais utilizadas para o período")

■ O volume de credenciais com alerta BLOCKED representa 0.51 do volume total de buscas por credenciais utilizadas para o período


TRIBUNAIS PRIORITÁRIOS

In [14]:
acompanhamento_de_projecao = pd.DataFrame(tabela_projecao)

# Filtrar as 3 maiores linhas da coluna "delta_credential_need"
n_maiores = 10
maiores_delta = acompanhamento_de_projecao.nlargest(n_maiores, 'delta_credential_need')[['environment', 'database_table', 'delta_credential_need']]



print(f"● Os 10 tribunais prioritários quanto à projeção de coleta [ref], para o período, foram:")
print(maiores_delta)

● Os 10 tribunais prioritários quanto à projeção de coleta [ref], para o período, foram:
  environment   database_table  delta_credential_need
0   JUSBRASIL      TJMG-PJE-2G                    396
1   JUSBRASIL         TRT2-PJE                    238
2   JUSBRASIL      TJDF-PJE-1G                    184
3   JUSBRASIL  TJPR-PROJUDI-1G                    151
4   JUSBRASIL  TJMG-PROJUDI-1G                     72
5   JUSBRASIL         TRT5-PJE                     64
6   JUSBRASIL         TRT7-PJE                     34
7   JUSBRASIL      TJDF-PJE-2G                     31
8   JUSBRASIL        TRT13-PJE                     19
9   JUSBRASIL      TST-PROP-2G                     18


CAPACIDADE DE USO

In [15]:
# Filtrar apenas as linhas onde delta_credential_need é maior que 0
filtrado_delta_maior_que_zero = acompanhamento_de_projecao[acompanhamento_de_projecao['delta_credential_need'] > 0]

filtrado_colunas = filtrado_delta_maior_que_zero[['database_table', 'environment', 'number_of_credentials', 'delta_credential_need']]

# Renomear as colunas
filtrado_colunas.rename(columns={
    'database_table': 'Tribunal',
    'environment': 'Pool',
    'number_of_credentials': 'Tínhamos',
    'delta_credential_need': 'Precisávamos'
}, inplace=True)

print(f"●	Tribunais que faltaram credenciais e quantas precisávamos para atender a demanda de Coletas:")
print(f"○ {filtrado_colunas}")

●	Tribunais que faltaram credenciais e quantas precisávamos para atender a demanda de Coletas:
○            Tribunal       Pool  Tínhamos  Precisávamos
0       TJMG-PJE-2G  JUSBRASIL      38.0           396
1          TRT2-PJE  JUSBRASIL       3.0           238
2       TJDF-PJE-1G  JUSBRASIL      26.0           184
3   TJPR-PROJUDI-1G  JUSBRASIL      26.0           151
4   TJMG-PROJUDI-1G  JUSBRASIL       2.0            72
5          TRT5-PJE  JUSBRASIL       2.0            64
6          TRT7-PJE  JUSBRASIL       3.0            34
7       TJDF-PJE-2G  JUSBRASIL      46.0            31
8         TRT13-PJE  JUSBRASIL       1.0            19
9       TST-PROP-2G  JUSBRASIL       2.0            18
10         TRT2-PJE    DIGESTO      15.0            14
11        TRT11-PJE  JUSBRASIL       1.0            14
12      TRF1-PJE-1G    DIGESTO       1.0             3
13         TRT7-PJE    DIGESTO       1.0             1


C:\Users\saman\AppData\Local\Temp\ipykernel_22280\2624466323.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtrado_colunas.rename(columns={


In [16]:
filtrado_ociosas = acompanhamento_de_projecao[
    (acompanhamento_de_projecao['number_of_credentials'] > 0) & 
    (acompanhamento_de_projecao['credential_total_need'] == 0)
]
filtrado_ociosas_df = pd.DataFrame(filtrado_ociosas)

filtrado_ociosas_df.rename(columns={
    'database_table': 'Tribunal',
    'environment': 'Pool'
}, inplace=True)

filtrado_ociosas_df_columns = filtrado_ociosas_df[['Tribunal', 'Pool']]

print("● Tribunais com credenciais ociosas*")
print(filtrado_ociosas_df_columns)

● Tribunais com credenciais ociosas*
            Tribunal       Pool
18       STF-PROP-2G  JUSBRASIL
24      TJCE-ESAJ-1G  JUSBRASIL
26       TRF3-PJE-2G  JUSBRASIL
27       TRF3-PJE-1G  JUSBRASIL
30       TST-PROP-2G    DIGESTO
31     TRF6-EPROC-1G  JUSBRASIL
32     TRF6-EPROC-2G  JUSBRASIL
36     TRF4-EPROC-2G    DIGESTO
37     TRF2-EPROC-2G    DIGESTO
39      TJCE-ESAJ-1G    DIGESTO
42        TST-PJE-2G    DIGESTO
43      TJAL-ESAJ-1G    DIGESTO
48      TJAC-ESAJ-1G    DIGESTO
50       TJBA-PJE-1G    DIGESTO
52       TJMA-PJE-2G    DIGESTO
53       TJPE-PJE-1G    DIGESTO
59       TJCE-PJE-2G  JUSBRASIL
60       TJCE-PJE-1G  JUSBRASIL
61     TRF6-EPROC-2G    DIGESTO
62     TRF6-EPROC-1G    DIGESTO
63       TJPE-PJE-2G    DIGESTO
66       TJAP-PJE-2G    DIGESTO
67       TJES-PJE-2G    DIGESTO
68       TJAP-PJE-1G    DIGESTO
70       TJPB-PJE-1G    DIGESTO
72       TJMA-PJE-1G    DIGESTO
74       TJPA-PJE-1G    DIGESTO
78       TJPB-PJE-2G    DIGESTO
79       TJRN-PJE-1G    DIGESTO
80 

CHECAGEM DE ALERTAS BLOCKED

○ Quantidade de alertas para o período de tempo determinado

In [17]:
# Converte a coluna "Data de bloqueio" para o tipo datetime
tabela['Data do bloqueio'] = pd.to_datetime(tabela['Data do bloqueio'], format='%d/%m/%Y')

# Filtre o DataFrame para o intervalo de datas
tabela_filtrada = tabela[(tabela['Data do bloqueio'] >= data_inicio) & (tabela['Data do bloqueio'] <= data_fim)]

# Obtenha a quantidade de linhas no DataFrame filtrado
qtd_bloqueios_atual = len(tabela_filtrada)

# Certifique-se de que data_inicio e data_fim sejam do tipo datetime
data_inicio = pd.to_datetime(data_inicio)
data_fim = pd.to_datetime(data_fim)

# Use .date() apenas após a conversão para datetime
print(f'Registramos {qtd_bloqueios_atual} alertas BLOCKED para o período de {data_inicio.date()} a {data_fim.date()}')


Registramos 361 alertas BLOCKED para o período de 2024-07-08 a 2024-07-14


○ Filtrando os bloqueios da semana anterior para comparação

In [18]:
#Filtrando os bloqueios da semana anterior para comparação

# Filtre o DataFrame para o intervalo de datas
tabela_filtrada_antiga = tabela[(tabela['Data do bloqueio'] >= data_inicio_antiga) & (tabela['Data do bloqueio'] <= data_fim_antiga)]


qtd_bloqueios_semana_anterior = len(tabela_filtrada_antiga)

print(f'Quantidade de bloqueios da semana passada: {qtd_bloqueios_semana_anterior}')



Quantidade de bloqueios da semana passada: 172


○ Comparação com a semana anterior

In [19]:

#Comparação entre semana anterior e semana Atual
if qtd_bloqueios_atual < qtd_bloqueios_semana_anterior:
    maior = qtd_bloqueios_semana_anterior
    menor= qtd_bloqueios_atual
    status = "redução"
    
elif qtd_bloqueios_atual > qtd_bloqueios_semana_anterior:
    menor = qtd_bloqueios_semana_anterior
    maior= qtd_bloqueios_atual
    status = "aumento"

comparação_entre_semanas = round((maior - menor)/maior*100, 2) #round para formatar em apenas 2 casas decimais

print(f'Houve {status} de {comparação_entre_semanas} em relação à semana anterior, em que tivemos {qtd_bloqueios_semana_anterior} alertas')

Houve aumento de 52.35 em relação à semana anterior, em que tivemos 172 alertas


○ Dia com maior número de alertas

In [20]:
#Dia com maior numero de alertas
# Contar a frequência de cada data
frequencia_datas = tabela_filtrada['Data do bloqueio'].value_counts() #value_counts() retorna uma Series com as datas como índices e as contagens como valores.

# Encontrar o dia com o maior número de repetições
dia_mais_repetido = frequencia_datas.idxmax() #idxmax() retorna o índice (data) com a maior contagem 
quantidade_repeticoes = frequencia_datas.max() #max() retorna o valor máximo (contagem).

print(f'O dia com o maior número de alertas foi: {dia_mais_repetido.date()}')
print(f'que contou com {quantidade_repeticoes} alertas de bloqueio reativo')

O dia com o maior número de alertas foi: 2024-07-10
que contou com 112 alertas de bloqueio reativo


○ Quantidade de alerta internos e externos

In [21]:
#Quantidade de alerta internos e externos
contagem_tipos = tabela_filtrada['Tipo Parceiro'].value_counts() #value_counts() retorna uma Series com os valores únicos na coluna e suas respectivas contagens.

contagem_interno = contagem_tipos.get('Interno', 0)
contagem_externo = contagem_tipos.get('Externo', 0)

print(f'{contagem_externo} alertas para externos e {contagem_interno} para internos')

213 alertas para externos e 148 para internos


○ Alertas para pool Jusbrasil e para pool Digesto

In [22]:
# Contar a frequência de cada valor na coluna "Pool"
contagem_pools = tabela_filtrada['Pool'].value_counts() #value_counts() retorna uma Series com os valores únicos da coluna como índices e suas respectivas contagens como valores.

# Contar o número de vezes que 'digesto' aparece
contagem_digesto = contagem_pools.get('digesto', 0)

# Contar o número de vezes que todos os outros valores aparecem
total_outros = contagem_pools.sum() - contagem_digesto

print(f'{total_outros} alertas para pool Jusbrasil e {contagem_digesto} para pool Digesto')


361 alertas para pool Jusbrasil e 0 para pool Digesto


○ Top 3 parceiros com maior número de alertas

In [23]:
#Top 3 parceiros com maior número de alertas
# Contar a frequência de cada e-mail na coluna "E-mail Parceiro (KM)"
contagem_bloqueios = tabela_filtrada['E-mail Parceiro (KM)'].value_counts()

# Obter os top 3 e-mails mais frequentes
top_3_emails = contagem_bloqueios.head(3) #head(3) retorna os 3 e-mails mais frequentes.

# Imprimir os top 3 e-mails e suas contagens
print('Top 3 parceiros com maior número de alertas:')
print(top_3_emails)



Top 3 parceiros com maior número de alertas:
E-mail Parceiro (KM)
adv.mrl.ba@gmail.com                   19
limasantanadandaradejesus@gmail.com    13
dosreiscaio7@gmail.com                 13
Name: count, dtype: int64


○ ATIVAS

In [24]:
#ATIVAS
quantidade_ativa = tabela_filtrada['Situação da credencial no Tribunal? '].value_counts().get('Ativa', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.

total_de_reports = tabela_filtrada['Situação da credencial no Tribunal? '].count()
porcentagem = round((quantidade_ativa*100)/total_de_reports,2)


print(f'{quantidade_ativa} credenciais apresentaram alerta de bloqueio no KM, mas estavam ativas no tribunal, representando {porcentagem}% do total de reports para o período.')

269 credenciais apresentaram alerta de bloqueio no KM, mas estavam ativas no tribunal, representando 74.52% do total de reports para o período.


○ COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA - ATIVAS

In [25]:
#COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA
quantidade_ativa_antiga = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].value_counts().get('Ativa', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.

total_de_reports_antigos = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].count()

if quantidade_ativa < quantidade_ativa_antiga:
    maior = quantidade_ativa_antiga
    menor= quantidade_ativa
    status_ativas = "uma redução"
    
elif quantidade_ativa > quantidade_ativa_antiga:
    menor = quantidade_ativa_antiga
    maior= quantidade_ativa
    status_ativas = "um aumento"


comparação_entre_semanas_ativa = round((maior - menor)/maior*100, 2)


print(f'Credenciais ativas atual: {quantidade_ativa}')
print(f'Credenciais ativas antiga: {quantidade_ativa_antiga}')
print(f'Houve {status_ativas} de {comparação_entre_semanas_ativa} em relação à semana anterior, em que tivemos {quantidade_ativa_antiga} situação da credencial no Tribunal = ativa ')


Credenciais ativas atual: 269
Credenciais ativas antiga: 71
Houve um aumento de 73.61 em relação à semana anterior, em que tivemos 71 situação da credencial no Tribunal = ativa 


○ Maiores ofensores

In [26]:
#Maiores ofensores
# Contar a frequência de cada valor na coluna
contagem_tribunal = tabela_filtrada['Tribunal/Sistema/Grau'].value_counts()

# Obter os top 5 valores mais frequentes
top_5_tribunais = contagem_tribunal.head(5)

# Imprimir os top 5 valores e suas contagens
print(f'Dentre esses “falsos alertas” de bloqueio, os maiores ofensores foram:')
print(top_5_tribunais)


Dentre esses “falsos alertas” de bloqueio, os maiores ofensores foram:
Tribunal/Sistema/Grau
TJMS-ESAJ-1G     107
TRT2-PJE          44
TJTO-EPROC-2G     18
TJMG-PJE-1G       15
TRT11-PJE         14
Name: count, dtype: int64


○ Ordenando o top 10 tribunais prioritarios

In [27]:
#Ordenando o top 10 tribunais prioritarios

ordenado = tabela_projecao.sort_values(by='delta_credential_need', ascending=False) #Isso organiza os dados em ordem decrescente com base na coluna "delta_credential_need".

# Selecionar os top 10 registros com maior valor em "delta_credential_need"
credenciais_prioritarias = ordenado.head(10)

# Selecionar apenas as colunas "database_table" e "delta_credential_need"
credenciais_prioritarias_filtrado = credenciais_prioritarias[['database_table', 'delta_credential_need']]

# Imprimir os resultados
print('Top 10 tribunais prioritários (com o maior "delta_credential_need")')
print(credenciais_prioritarias_filtrado)



Top 10 tribunais prioritários (com o maior "delta_credential_need")
    database_table  delta_credential_need
0      TJMG-PJE-2G                    396
1         TRT2-PJE                    238
2      TJDF-PJE-1G                    184
3  TJPR-PROJUDI-1G                    151
4  TJMG-PROJUDI-1G                     72
5         TRT5-PJE                     64
6         TRT7-PJE                     34
7      TJDF-PJE-2G                     31
8        TRT13-PJE                     19
9      TST-PROP-2G                     18


In [28]:
#Filtrando apenas as com situação ativa
tabela_filtrada_ativas = tabela_filtrada[tabela_filtrada['Situação da credencial no Tribunal? '] == 'Ativa']
tabela_filtrada_ativas_antiga = tabela_filtrada_antiga[tabela_filtrada_antiga['Situação da credencial no Tribunal? '] == 'Ativa']

○ Vendo se as credenciais dos tribunais prioritários estão no acompanhamento atual

In [29]:
#Vendo se as credenciais dos tribunais prioritários estão no acompanhamento atual
# Corrigir o nome das colunas se necessário
credenciais_prioritarias_filtrado.columns = credenciais_prioritarias_filtrado.columns.str.strip()
tabela_filtrada_ativas.columns = tabela_filtrada_ativas.columns.str.strip()

# Nome das colunas com dados a serem comparados
coluna_prioritaria = 'database_table'
coluna_ativa = 'Tribunal/Sistema/Grau'



# Obter a lista de credenciais prioritárias
credenciais = credenciais_prioritarias_filtrado[coluna_prioritaria].unique()

resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_ativas[tabela_filtrada_ativas[coluna_ativa] == credencial].shape[0]
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})

# Converter a lista de resultados em um DataFrame
resultado_df = pd.DataFrame(resultados)

# Ordenar o resultado pela quantidade em ordem decrescente
resultado_df = resultado_df.sort_values(by='quantidade', ascending=False)

# Imprimir o resultado
print(resultado_df)

        Tribunais:  quantidade
1         TRT2-PJE          14
2      TJDF-PJE-1G           5
7      TJDF-PJE-2G           5
0      TJMG-PJE-2G           0
4  TJMG-PROJUDI-1G           0
3  TJPR-PROJUDI-1G           0
5         TRT5-PJE           0
6         TRT7-PJE           0
8        TRT13-PJE           0
9      TST-PROP-2G           0


○ Vendo se as credenciais dos tribunais prioritários estão no acompanhamento da semana passada

In [30]:
#Vendo se as credenciais dos tribunais prioritários estão no acompanhamento da semana passada
# Nome das colunas com dados a serem comparados
coluna_prioritaria = 'database_table'
coluna_ativa = 'Tribunal/Sistema/Grau'



# Obter a lista de credenciais prioritárias
credenciais = credenciais_prioritarias_filtrado[coluna_prioritaria].unique()

resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_ativas_antiga[tabela_filtrada_ativas_antiga[coluna_ativa] == credencial].shape[0] # .shape() usado para obter o número de linhas de um DataFrame
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})

# Converter a lista de resultados em um DataFrame
resultado_df2 = pd.DataFrame(resultados)

# Ordenar o resultado pela quantidade em ordem decrescente
resultado_df_antigo = resultado_df2.sort_values(by='quantidade', ascending=False)

print("Em relação a semana passada:")
print("")
print(resultado_df_antigo)

Em relação a semana passada:

        Tribunais:  quantidade
7      TJDF-PJE-2G           4
0      TJMG-PJE-2G           0
1         TRT2-PJE           0
2      TJDF-PJE-1G           0
4  TJMG-PROJUDI-1G           0
3  TJPR-PROJUDI-1G           0
5         TRT5-PJE           0
6         TRT7-PJE           0
8        TRT13-PJE           0
9      TST-PROP-2G           0


○

In [31]:
# Verificar se os DataFrames têm o mesmo número de linhas
if resultado_df.shape[0] != resultado_df_antigo.shape[0]:
    raise ValueError("Os DataFrames não têm o mesmo número de linhas. Alinhe-os antes de comparar.")

# Verificar se as colunas existem em ambos os DataFrames
if 'quantidade' not in resultado_df.columns or 'quantidade' not in resultado_df_antigo.columns:
    raise ValueError("A coluna 'quantidade' não foi encontrada em um dos DataFrames.")

# Calcular a diferença percentual entre quantidade atual e passada
def calcular_diferenca_percentual(atual, passada):
    if atual == 0 and passada == 0:
        return 0  # Evitar divisão por zero
    elif atual == 0:
        return -100  # Se atual é 0, a diferença percentual é -100%
    elif passada == 0:
        return 100  # Se passada é 0, a diferença percentual é 100%
    else:
        maior_num = max(atual, passada)
        menor_num = min(atual, passada)
        return round((maior_num - menor_num) / maior_num * 100, 2)
        

# Comparar linha por linha
comparacao = pd.DataFrame({
    'Tribunais': resultado_df['Tribunais:'],
    'quantidade_passada': resultado_df_antigo['quantidade'].values,
    'quantidade_atual': resultado_df['quantidade'],
    'diferenca_percentual': [
        calcular_diferenca_percentual(atual, passada)
        for atual, passada in zip(resultado_df['quantidade'], resultado_df_antigo['quantidade'])
    ]
})

print(comparacao)

         Tribunais  quantidade_passada  quantidade_atual  diferenca_percentual
1         TRT2-PJE                   4                14                 71.43
2      TJDF-PJE-1G                   0                 5                100.00
7      TJDF-PJE-2G                   0                 5                100.00
0      TJMG-PJE-2G                   0                 0                  0.00
4  TJMG-PROJUDI-1G                   0                 0                  0.00
3  TJPR-PROJUDI-1G                   0                 0                  0.00
5         TRT5-PJE                   0                 0                  0.00
6         TRT7-PJE                   0                 0                  0.00
8        TRT13-PJE                   0                 0                  0.00
9      TST-PROP-2G                   0                 0                  0.00


○ BLOQUEADAS

In [32]:
#BLOQUEADAS
quantidade_bloqueada = tabela_filtrada['Situação da credencial no Tribunal? '].value_counts().get('Bloqueada', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Bloqueada', 0) retorna a contagem de "Bloqueada" ou 0 se "Bloqueada" não estiver presente.

total_de_reports = tabela_filtrada['Situação da credencial no Tribunal? '].count()
porcentagem_bloqueadas = round((quantidade_bloqueada*100)/total_de_reports,2)


print(f'{quantidade_bloqueada} credenciais foi efetivamente bloqueada, representando {porcentagem_bloqueadas}% do total de reports para o período. .')

11 credenciais foi efetivamente bloqueada, representando 3.05% do total de reports para o período. .


○ COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA - BLOQUEADAS

In [33]:

#COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA
quantidade_bloqueadas_antiga = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].value_counts().get('Bloqueada', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.



total_de_reports_antigos = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].count()

if quantidade_bloqueada < quantidade_bloqueadas_antiga:
    maior = quantidade_bloqueadas_antiga
    menor= quantidade_bloqueada
    status_bloqueadas = "uma redução"
    
elif quantidade_bloqueada > quantidade_bloqueadas_antiga:
    menor = quantidade_bloqueadas_antiga
    maior= quantidade_bloqueada
    status_bloqueadas = "um aumento"
tabela_filtrada_antiga

comparação_entre_semanas_bloqueada = round((maior - menor)/maior*100, 2)
#

print(f'Credenciais bloqueadas Atual: {quantidade_bloqueada}')
print(f'Credenciais bloqueadas Antiga: {quantidade_bloqueadas_antiga}')
print(f'Houve {status_bloqueadas} de {comparação_entre_semanas_bloqueada}% na taxa de bloqueios, em relação à semana anterior, representada por {quantidade_bloqueadas_antiga} alertas  de situação da credencial no Tribunal = bloqueada')




print(coluna_ativa)



Credenciais bloqueadas Atual: 11
Credenciais bloqueadas Antiga: 8
Houve um aumento de 27.27% na taxa de bloqueios, em relação à semana anterior, representada por 8 alertas  de situação da credencial no Tribunal = bloqueada
Tribunal/Sistema/Grau



○ COM RELAÇÃO AOS TRIBUNAIS PRIORITÁRIOS - BLOQUEADAS

In [34]:
#COM RELAÇÃO AOS TRIBUNAIS PRIORITÁRIOS
#credenciais_prioritarias_filtrado2 = credenciais_prioritarias_filtrado['database_table']

# Obter a lista de credenciais prioritárias
credenciais = credenciais_prioritarias_filtrado[coluna_prioritaria].unique()

tabela_filtrada_bloqueada = tabela_filtrada[tabela_filtrada['Situação da credencial no Tribunal? '] == 'Bloqueada']



resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_bloqueada[tabela_filtrada_bloqueada[coluna_ativa] == credencial].shape[0] # .shape() usado para obter o número de linhas de um DataFrame
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})




if contagem == 0:
    print("Com relação aos tribunais prioritários:")
    resultado_df_antigo = "não tivemos alertas de bloqueios."
    print(f"{resultado_df_antigo}")
else:
    # Converter a lista de resultados em um DataFrame
    resultado_df2 = pd.DataFrame(resultados)

    # Ordenar o resultado pela quantidade em ordem decrescente
    resultado_df_antigo = resultado_df2.sort_values(by='quantidade', ascending=False)
    print("Com relação aos tribunais prioritários:")
    print("")
    print(resultado_df_antigo)



Com relação aos tribunais prioritários:
não tivemos alertas de bloqueios.


○ INATIVAS

In [35]:
quantidade_inativa = tabela_filtrada['Situação da credencial no Tribunal? '].value_counts().get('Inativa', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Bloqueada', 0) retorna a contagem de "Bloqueada" ou 0 se "Bloqueada" não estiver presente.

porcentagem_inativas = round((quantidade_inativa*100)/total_de_reports,2)

print(f'{quantidade_inativa} credenciais foram inativadas no período, representando {porcentagem_inativas} do total de reports para o período')

65 credenciais foram inativadas no período, representando 18.01 do total de reports para o período


○ COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA - INATIVAS

In [36]:
#COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA
quantidade_inativa_antiga = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].value_counts().get('Inativa', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.

total_de_reports_antigos = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].count()

if quantidade_inativa < quantidade_inativa_antiga:
    maior = quantidade_inativa_antiga
    menor= quantidade_inativa
    status_inativa = "uma redução"
    
elif quantidade_inativa > quantidade_inativa_antiga:
    menor = quantidade_inativa_antiga
    maior= quantidade_inativa
    status_inativa = "um aumento"

comparação_entre_semanas_inativas = round((maior - menor)/maior*100, 2)


print(f'Credenciais inativas Atual: {quantidade_inativa}')
print(f'Credenciais inativas Antiga: {quantidade_inativa_antiga}')
print(f'Houve {status_inativa} de {comparação_entre_semanas_inativas}% com relação à semana anterior, em que tivemos {quantidade_inativa_antiga} alertas  de situação da credencial no Tribunal = Inativa')






Credenciais inativas Atual: 65
Credenciais inativas Antiga: 72
Houve uma redução de 9.72% com relação à semana anterior, em que tivemos 72 alertas  de situação da credencial no Tribunal = Inativa


○ COM RELAÇÃO AOS TRIBUNAIS PRIORITÁRIOS - INATIVAS

In [37]:

tabela_filtrada_inativa = tabela_filtrada[tabela_filtrada['Situação da credencial no Tribunal? '] == 'Inativa']

resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_inativa[tabela_filtrada_inativa[coluna_ativa] == credencial].shape[0] # .shape() usado para obter o número de linhas de um DataFrame
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})
    




if contagem == 0:
    print("Com relação aos tribunais prioritários:")
    resultado_df3 = "não tivemos alertas de inativas."
    print(f"{resultado_df3}")

else:
    # Converter a lista de resultados em um DataFrame
    resultado_df3 = pd.DataFrame(resultados)


    # Ordenar o resultado pela quantidade em ordem decrescente
    resultado_df3 = resultado_df3.sort_values(by='quantidade', ascending=False)
    print("Com relação aos tribunais prioritários, tivemos:")
    print(resultado_df3)



Com relação aos tribunais prioritários:
não tivemos alertas de inativas.


○	INCORRETAS

In [38]:
quantidade_incorreta = tabela_filtrada['Situação da credencial no Tribunal? '].value_counts().get('Incorreta', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Bloqueada', 0) retorna a contagem de "Bloqueada" ou 0 se "Bloqueada" não estiver presente.

porcentagem_incorrentas = round((quantidade_incorreta*100)/total_de_reports,2)


print(f"{quantidade_incorreta} credenciais tiveram alerta para senha inválida/incorreta, representando {porcentagem_incorrentas}% do total de reports para o período, que foi {total_de_reports}.")

9 credenciais tiveram alerta para senha inválida/incorreta, representando 2.49% do total de reports para o período, que foi 361.


○ COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA - INCORRETAS

In [39]:
#COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA
quantidade_incorreta_antiga = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].value_counts().get('Incorreta', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.

total_de_reports_antigos = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].count()

if quantidade_incorreta < quantidade_incorreta_antiga:
    maior = quantidade_incorreta_antiga
    menor= quantidade_incorreta
    status_incorreta = "uma redução"
    
elif quantidade_incorreta > quantidade_incorreta_antiga:
    menor = quantidade_incorreta_antiga
    maior= quantidade_incorreta
    status_incorreta = "um aumento"

comparação_entre_semanas_incorreta = round((maior - menor)/maior*100, 2)


print(f'Credenciais incorretas Atual: {quantidade_incorreta}')
print(f'Credenciais incorretas Antiga: {quantidade_incorreta_antiga}')
print(f'Houve {status_incorreta} de {comparação_entre_semanas_incorreta}% com relação à semana anterior, em que tivemos {quantidade_incorreta_antiga} alertas  de situação da credencial no Tribunal = Incorreta')


Credenciais incorretas Atual: 9
Credenciais incorretas Antiga: 13
Houve uma redução de 30.77% com relação à semana anterior, em que tivemos 13 alertas  de situação da credencial no Tribunal = Incorreta


○ COM RELAÇÃO AOS TRIBUNAIS PRIORITÁRIOS - INCORRETAS

In [40]:

# Obter a lista de credenciais prioritárias
tabela_filtrada_incorreta = tabela_filtrada[tabela_filtrada['Situação da credencial no Tribunal? '] == 'Incorreta']

resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_incorreta[tabela_filtrada_incorreta[coluna_ativa] == credencial].shape[0] # .shape() usado para obter o número de linhas de um DataFrame
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})
    




if contagem == 0:
    print("Com relação aos tribunais prioritários:")
    resultado_df4 = "não tivemos alertas de incorretas."
    print(resultado_df4)

else:
    # Converter a lista de resultados em um DataFrame
    resultado_df3 = pd.DataFrame(resultados)
    # Ordenar o resultado pela quantidade em ordem decrescente
    resultado_df4 = resultado_df3.sort_values(by='quantidade', ascending=False)

    print("Com relação aos tribunais prioritários, tivemos:")
    print("")
    print(resultado_df4)


Com relação aos tribunais prioritários:
não tivemos alertas de incorretas.


○	VENCIDAS

In [41]:
quantidade_vencidas = tabela_filtrada['Situação da credencial no Tribunal? '].value_counts().get('Vencida/Expirada', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Bloqueada', 0) retorna a contagem de "Bloqueada" ou 0 se "Bloqueada" não estiver presente.

porcentagem_vencidas = round((quantidade_vencidas*100)/total_de_reports,2)


print(f"{quantidade_vencidas} credenciais tiveram alerta para senha vencidas, representando {porcentagem_vencidas}% do total de reports para o período, que foi {total_de_reports}.")

7 credenciais tiveram alerta para senha vencidas, representando 1.94% do total de reports para o período, que foi 361.


○ COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA - VENCIDAS

In [42]:
#COMPARAÇÃO EM RELAÇÃO A SEMANA PASSADA
quantidade_vencidas_antiga = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].value_counts().get('Vencida/Expirada', 0) #value_counts() conta o número de vezes que cada valor aparece na coluna. get('Ativa', 0) retorna a contagem de "Ativa" ou 0 se "Ativa" não estiver presente.

total_de_reports_antigos = tabela_filtrada_antiga['Situação da credencial no Tribunal? '].count()

if quantidade_vencidas < quantidade_vencidas_antiga:
    maior = quantidade_vencidas_antiga
    menor= quantidade_vencidas
    status_vencidas = "uma redução"
    
elif quantidade_vencidas > quantidade_vencidas_antiga:
    menor = quantidade_vencidas_antiga
    maior= quantidade_vencidas
    status_vencidas = "um aumento"

elif quantidade_vencidas == quantidade_vencidas_antiga:
    status_vencidas = "não mudou"

comparação_entre_semanas_vencida = round((maior - menor)/maior*100, 2)


print(f'Credenciais vencidas Atual: {quantidade_vencidas}')
print(f'Credenciais vencidas Antiga: {quantidade_vencidas_antiga}')
print(f'Houve {status_vencidas} de {comparação_entre_semanas_vencida}% com relação à semana anterior, em que tivemos {quantidade_vencidas_antiga} alertas  de situação da credencial no Tribunal = vencida')


Credenciais vencidas Atual: 7
Credenciais vencidas Antiga: 7
Houve não mudou de 30.77% com relação à semana anterior, em que tivemos 7 alertas  de situação da credencial no Tribunal = vencida


○ COM RELAÇÃO AOS TRIBUNAIS PRIORITÁRIOS - VENCIDAS

In [43]:
# Obter a lista de credenciais prioritárias
tabela_filtrada_vencidas = tabela_filtrada[tabela_filtrada['Situação da credencial no Tribunal? '] == 'Vencida/Expirada']

resultados = []

# Contar quantas vezes cada credencial prioritária aparece na tabela de credenciais ativas
for credencial in credenciais:
    contagem = tabela_filtrada_vencidas[tabela_filtrada_vencidas[coluna_ativa] == credencial].shape[0] # .shape() usado para obter o número de linhas de um DataFrame
    resultados.append({'Tribunais:': credencial, 'quantidade': contagem})


if contagem == 0:
    print("Com relação aos tribunais prioritários:")
    resultado_df5 ="não tivemos alertas de vencidas."
    print(resultado_df5)

else:
    # Converter a lista de resultados em um DataFrame
    resultado_df4 = pd.DataFrame(resultados)
    # Ordenar o resultado pela quantidade em ordem decrescente
    resultado_df5 = resultado_df4.sort_values(by='quantidade', ascending=False)
    print("Com relação aos tribunais prioritários, tivemos:")
    
    print(resultado_df5)

Com relação aos tribunais prioritários:
não tivemos alertas de vencidas.


In [44]:
import sys
print(sys.executable)


c:\Users\saman\Documents\Pastas_Jus_Analise\Fup_Automatico\novo_venv\Scripts\python.exe


○ INTEGRAÇÃO COM GOOGLE SHEETS

○ TRANSFORMANDO OS DADOS EM DOC


In [45]:


from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT


    
# Função para criar o DOCX
def create_docx(filename):
    doc = Document()

    # Mensagem inicial
    intro = doc.add_paragraph(f"Bom dia, pessoal! Tudo bem? Atualizações sobre o comportamento das credenciais no período de {data_inicio} a {data_fim}:")
    intro.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    

    # Seção: Quantidade de credenciais
    credenciais_title = doc.add_heading("Quantidade de credenciais", level=2)
    

    credenciais_info = [
        f"● No período analisado, utilizamos {total_credenciais} credenciais diferentes (únicas) [ref]",
        f"● {quantidade_linhas_unicas} credenciais diferentes receberam alerta BLOCKED pelo KM, representando {porcentagem_credenciais} do total de credenciais utilizadas",
        f"● {quantidade_linhas_domains} domains requisitaram credenciais",
        f"○ Domains que mais requisitaram credenciais para o período*:",
        f"■ {resultado}",
        f"*Obs: listagem não exaustiva, inserimos aqui apenas o top 10 domains que mais requisitaram credenciais. A listagem completa de quanto cada domain requisitou está aqui."
    ]
    
    for line in credenciais_info:
        doc.add_paragraph(line)

    # Seção: Buscas e sessões
    buscas_title = doc.add_heading("Buscas e sessões", level=2)

    buscas_info = [
        f"● Nossos robôs buscaram {total_buscas} vezes por credenciais e com elas realizaram {total_realizadas} sessões de uso para o período analisado [ref]",
        f"○ {total_buscas_success} sessões tiveram sucesso, correspondendo a {relação_ao_total_success} do total.",
        f"○{total_buscas_error} sessões tiveram erros, correspondendo a {relação_ao_total_error} do total.",
        f"■ Dessas sessões com erro, tivemos {total_buscas_two_factor} sessões com erro no 2FA para:",
        f"{valores_domain_alias}",
        "",
        f"■ Tivemos {quantidade_linhas_blocked} alertas BLOCKED de credenciais realizados pelo KM [ref]",
        f"■ O volume de credenciais com alerta BLOCKED representa {relação_blocked_total} do volume total de buscas por credenciais utilizadas para o período"
    ]
    
    for line in buscas_info:
        doc.add_paragraph(line)

    # Seção: Tribunais prioritários
    tribunais_title = doc.add_heading("Tribunais prioritários", level=2)


    tribunais_info = [
        "● Os 10 tribunais prioritários quanto à projeção de coleta [ref], para o período, foram:",
        f"○ {maiores_delta}",
    
    ]

    for line in tribunais_info:
        doc.add_paragraph(line)

    # Seção: Capacidade de uso
    capacidade_title = doc.add_heading("Capacidade de uso [ref]", level=2)

    capacidade_info = [
        "● Tribunais que faltaram credenciais e quantas precisávamos para atender a demanda de coletas:",
        f"○ {filtrado_colunas}"
    ]

    for line in capacidade_info:
        doc.add_paragraph(line)

    # Seção: Tribunais com credenciais ociosas
    ociosas_title = doc.add_heading("● Tribunais com credenciais ociosas*", level=2)

    ociosas_info = [
        f"○ {filtrado_ociosas_df_columns} "
    ]

    for line in ociosas_info:
        doc.add_paragraph(line)

    # Observação final
    obs = doc.add_paragraph("*Obs: listagem não exaustiva. Inserimos aqui apenas domains que tínhamos credenciais, mas não precisávamos de nenhuma para o período. A listagem completa, com os valores, estão aqui.")

    # Título
    title = doc.add_heading("Checagem de alertas BLOCKED", level=1)

    # Análises da checagem
    doc.add_heading("● Análises da checagem", level=2)

    # Adicionar análises
    analyses = [
        f"● Registramos {qtd_bloqueios_atual} alertas BLOCKED para o período de {data_inicio} a {data_fim}",
        f"● Houve {status} de {comparação_entre_semanas} em relação à semana anterior, em que tivemos {qtd_bloqueios_semana_anterior} alertas",
        f"● O dia com o maior número de alertas foi {dia_mais_repetido.date()}, que contou com {quantidade_repeticoes} alertas de bloqueio reativo",
        f"● {contagem_externo} alertas para externos e {contagem_interno} para internos",
        f"● {total_outros} alertas para pool Jusbrasil e {contagem_digesto} para pool Digesto"
    ]
    for text in analyses:
        doc.add_paragraph(text)
        

    # Top 3 parceiros
    doc.add_heading("● Top 3 parceiros com maior número de alertas:", level=2)
    

    # Criar um DataFrame a partir da série
    df = top_3_emails.reset_index()  # Transforma a série em DataFrame
    df.columns = ['E-mail Parceiro (KM)', 'Contagem']  # Renomeia as colunas

    # Adicionar a tabela ao documento
    table = doc.add_table(rows=1, cols=2)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = "E-mail Parceiro (KM)"
    hdr_cells[1].text = "Contagem"
    
    for index, row in df.iterrows():
        row_cells = table.add_row().cells
        row_cells[0].text = row['E-mail Parceiro (KM)']
        row_cells[1].text = str(row['Contagem'])

    # Estilo da tabela
    for cell in table.rows[0].cells:
        cell.paragraphs[0].runs[0].font.bold = True


    # Seções adicionais
    doc.add_heading("Ativas", level=2)
    active_analysis = [
        f"■ {quantidade_ativa} credenciais apresentaram alerta de bloqueio no KM, mas estavam ativas no tribunal, representando {porcentagem}% do total de reports para o período.",
        f"■ Houve {status_ativas} de {comparação_entre_semanas_ativa} em relação à semana anterior, em que tivemos {quantidade_ativa_antiga} situação da credencial no Tribunal = ativa ",
        f"■	Com relação aos tribunais prioritários do período, tivemos:",
        f"{credenciais_prioritarias_filtrado}",
        ""
        f"■ Dentre esses “falsos alertas” de bloqueio, os maiores ofensores foram:"
    ]
    
    for analysis in active_analysis:
        doc.add_paragraph(analysis)
      
    
    # Criar um DataFrame a partir da série
    df = top_5_tribunais.reset_index()  # Transforma a série em DataFrame
    df.columns = ['Tribunal/Sistema/Grau', 'Contagem']  # Renomeia as colunas

    # Adicionar a tabela ao documento
    table = doc.add_table(rows=1, cols=2)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = "Tribunal/Sistema/Grau"
    hdr_cells[1].text = "Contagem"

    for index, row in df.iterrows():
        row_cells = table.add_row().cells
        row_cells[0].text = row['Tribunal/Sistema/Grau']
        row_cells[1].text = str(row['Contagem'])

    # Estilo da tabela
    for cell in table.rows[0].cells:
        cell.paragraphs[0].runs[0].font.bold = True

    doc.add_paragraph()  # Espaço

    

    # Exemplo para bloqueadas
    doc.add_heading("Bloqueadas", level=2)
    blocked_info = [
        f"■ {quantidade_bloqueada} credenciais foram efetivamente bloqueada, representando {porcentagem_bloqueadas}% do total de reports para o período.",
        f"■ Houve {status_bloqueadas} de {comparação_entre_semanas_bloqueada}% na taxa de bloqueios, em relação à semana anterior, representada por {quantidade_bloqueadas_antiga} alertas  de situação da credencial no Tribunal = bloqueada",
        f"■ Com relação aos tribunais prioritários: {resultado_df_antigo}."
    ]
    for info in blocked_info:
        doc.add_paragraph(info)
        doc.add_paragraph()  # Espaço


        # Seção Inativas
    doc.add_heading('Inativas', level=2)
    doc.add_paragraph(f'• {quantidade_inativa} credenciais foram inativadas no período, representando {porcentagem_inativas} do total de reports para o período.')
    doc.add_paragraph(f'• Houve {status_inativa} de {comparação_entre_semanas_inativas}% com relação à semana anterior, em que tivemos {quantidade_inativa_antiga} alertas.')
    doc.add_paragraph('• Com relação aos tribunais prioritários:')
    doc.add_paragraph(f'• {resultado_df3}')

    # Seção Incorretas
    doc.add_heading('Incorretas', level=2)
    doc.add_paragraph(f'• {quantidade_incorreta} credenciais tiveram alerta para senha inválida/incorreta, representando {porcentagem_incorrentas}% do total de reports para o período.')
    doc.add_paragraph(f'• Houve {status_incorreta} de {comparação_entre_semanas_incorreta}% com relação à semana anterior, em que tivemos {quantidade_incorreta_antiga} alertas  de situação da credencial no Tribunal = Incorreta')
    doc.add_paragraph(f'• Com relação aos tribunais prioritários:')
    doc.add_paragraph(f'• {resultado_df4}')

    # Seção Vencidas
    doc.add_heading('Vencidas', level=2)
    doc.add_paragraph(f'• {quantidade_vencidas} credenciais tiveram alerta para senha vencidas, representando {porcentagem_vencidas}% do total de reports para o período, que foi {total_de_reports}.')
    doc.add_paragraph(f'• Houve {status_vencidas} de {comparação_entre_semanas_vencida}% com relação à semana anterior, em que tivemos {quantidade_vencidas_antiga} alertas  de situação da credencial no Tribunal = vencida.')
    doc.add_paragraph('• Com relação aos tribunais prioritários:')
    doc.add_paragraph(f'• {resultado_df5}')

    # Finalizando o documento
    try:
        doc.save(filename)  # Sobrescreve o arquivo se já existir
        print(f'DOCX gerado com sucesso: {filename}')
    except PermissionError:
        print("Erro: O arquivo está aberto em outro programa ou você não tem permissão para sobrescrever.")

# Chamada da função com o caminho especificado
create_docx(r"C:\Users\saman\Downloads\Fup_semanal.docx")

DOCX gerado com sucesso: C:\Users\saman\Downloads\Fup_semanal.docx
